# multi layer nn
# digit classification based on MNIST

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
input_size = 784 # image size 28 x 28 = 784 when flatten
hidden_size = 100 # number of neurons
num_classes = 10
num_epochs = 5
batch_size = 100
lr = 0.001

In [9]:
train_dataset = torchvision.datasets.MNIST(root = "./data", 
                                           train = True, 
                                           transform = transforms.ToTensor(), download = False)
test_dataset = torchvision.datasets.MNIST(root = "./data", 
                                           train = False, 
                                           transform = transforms.ToTensor(), download = False)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

# check one batch

In [ ]:
examples = iter(train_loader)
samples, labels = examples.next()
samples.shape, labels.shape # batch size 100, 1 colour channel, image size 28x28

In [ ]:
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(samples[i][0], cmap = "gray") # index i sample and first colour channel
plt.show()

# model

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNeturalNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.hidden_sizeiznum_classesdden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

# training loop

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device) # 100, 1 , 28, 28 to 100, 784
        labels = labels.to(device)
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}")

# eval

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for images, labels in test_loader():
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # value, index
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
    acc = 100.0 * n_correct/n_samples
    acc